In [1]:
# კორპუსთან სამუშაო ბიბლიოთეკის იმპორტი
from GNLP.corpus import Corpus
corpus = Corpus()

2020-10-30 20:21:14,589	INFO -- MainProcess corpus.py:17 -- Welcome to the Georgian NLP toolset demo


In [2]:
# ვებ სკრაპი
corpus.from_file('data/corpuses/kat-ge_web_2019_1M/kat-ge_web_2019_1M-sentences.txt')
corpus.file2sequence()
len(corpus.sequence)

2020-10-30 20:21:14,595	INFO -- MainProcess corpus.py:113 -- ვიწყებ data/corpuses/kat-ge_web_2019_1M/kat-ge_web_2019_1M-sentences.txt-ის დამუშავებას
2020-10-30 20:21:15,715	INFO -- MainProcess corpus.py:126 -- წინადადების რაოდენობა: 1000000


1000000

In [3]:
# 2016 წლისთვის ვებ ნიუსები
corpus.from_file('data/corpuses/kat_newscrawl_2016_1M/kat_newscrawl_2016_1M-sentences.txt')
corpus.file2sequence()
len(corpus.sequence)


2020-10-30 20:21:15,785	INFO -- MainProcess corpus.py:113 -- ვიწყებ data/corpuses/kat_newscrawl_2016_1M/kat_newscrawl_2016_1M-sentences.txt-ის დამუშავებას
2020-10-30 20:21:17,048	INFO -- MainProcess corpus.py:126 -- წინადადების რაოდენობა: 2000000


2000000

In [4]:
# უახლესი ქართული ვიკიპედია
corpus.from_file('data/corpuses/kawiki-latest-pages-articles_preprocessed.txt')
corpus.file2sequence()
len(corpus.sequence)


2020-10-30 20:21:17,057	INFO -- MainProcess corpus.py:113 -- ვიწყებ data/corpuses/kawiki-latest-pages-articles_preprocessed.txt-ის დამუშავებას
2020-10-30 20:21:18,814	INFO -- MainProcess corpus.py:126 -- წინადადების რაოდენობა: 3657281


3657281

In [5]:
corpus.preprocess_sequence()
len(corpus.prepro_sequence)

3657281

In [6]:
with open('prepro.txt', 'w', encoding='utf-8') as file:
    for sentence in corpus.prepro_sequence:
        file.write(f"{sentence}\n")

In [19]:
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path("./").glob("*.txt")]

print(f'paths: {paths}')

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

paths: ['prepro.txt', 'requirements.txt']


In [20]:
# Save files to disk
tokenizer.save_model(".", "QartBERT")

['./QartBERT-vocab.json', './QartBERT-merges.txt']

In [21]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./QartBERT-vocab.json",
    "./QartBERT-merges.txt",
)
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

print(
    tokenizer.encode("ვინ ხარ შენ?")
)


Encoding(num_tokens=6, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In [24]:
from tokenizers.decoders import ByteLevel
decoder = ByteLevel()
toks = tokenizer.encode("ვინ ხარ შენ?").tokens

decoder.decode(toks)

'<s>ვინ ხარ შენ?</s>'

In [25]:
with open('prepro.txt', 'r', encoding='utf-8') as file:
    print(file.readlines()[0:10])

['დაემატა თქვენ კალათაში\n', 'მაისურაძე წიგნის\n', 'რიგის ტიპი პირველადი მეორეული შეფუთვის მარკირების დიზაინის სპეციფიკაციაში ცვლილების რეგისტრაცია\n', 'საათზე გაიხსნა უბნები ოპოზიციური პარტიებისგან გაკეთებული განცხადებები არის სიცრუე არის ტყუილი\n', 'ფინალურ ეტაპზე მეტოქეს ელოდება აჭარაბეთი რომელმაც ვითიბი ბანკი დაამარცხა\n', 'ქილის მოცულობა კუნელის ახალი ყვავილებით აავსეთ ბოლომდე არყით აავსეთ კვირა დააყენეთ\n', 'ჭიქა მარტივი სიროფი ჭიქა წყალი ჭიქა შაქარი\n', 'წამი ეწოდება მელოდრამას რომელშიც დავით ქაცარავა ლუიზა ნერსისიანი მონაწილეობენ\n', 'საათის მონაცემებით ამომრჩეველია ურნასთან მისული\n', 'უფასო ოპერაცია უსინათლოებს\n']


In [26]:
# Check that we have a GPU
!nvidia-smi

Fri Oct 30 20:54:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla M10           Off  | 00000000:3D:00.0 Off |                  N/A |
| N/A   30C    P0    16W /  53W |      0MiB /  8129MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla M10           Off  | 00000000:3E:00.0 Off |                  N/A |
| N/A   30C    P0    17W /  53W |      0MiB /  8129MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [33]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-wjfezc9l
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 883 kB 682 kB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 666 kB 10.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 10.0 MB/s eta 0:00:01
     |████████████████████████████████| 70 kB 10.9 MB/s eta 0:00:01
     |████████████████████████████████| 301 kB 11.3 MB/s eta 0:00:01
  Created wheel for transformers: filename=transformers-3.4.0-py3-none-any.whl size=1285604 sha256=b5ebf0752265e91093ec2ae0afae487438704300a713cce2715dbd1f5040cb3f
  Stored in directory: /tmp/pip-ephem-wheel-cache-ywu22d0a/wheels/5a/0a/d0/eb8d0ea1d7d02156f8675d6e5dfa52c03601cbe377290db8dc
  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893259 

In [1]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [2]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [3]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./QartBERT", max_len=512)

In [4]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [5]:
model.num_parameters()

83504416

In [6]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./prepro.txt",
    block_size=128,
)

CPU times: user 17min 1s, sys: 24.5 s, total: 17min 25s
Wall time: 1min 35s


In [7]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./QartBERT",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

/home/tchichua/anaconda3/envs/env3.6/lib/python3.6/site-packages/transformers/trainer.py:281: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead. Setting `args.prediction_loss_only=True
  FutureWarning,


In [9]:
%%time
trainer.train()
trainer.save_model("./QartBERT")

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
/home/tchichua/anaconda3/envs/env3.6/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


RuntimeError: CUDA out of memory. Tried to allocate 3.17 GiB (GPU 0; 7.94 GiB total capacity; 4.51 GiB already allocated; 237.62 MiB free; 7.25 GiB reserved in total by PyTorch)

In [ ]:
torch.cuda.empty_cache()